In [35]:
# -*- coding: utf-8 -*-

In [36]:
import requests
import pandas as pd
import json

In [37]:
def getDatumi():
    listaDatuma = []
    danaUMjesecu = {
        3:31,
        4:30,
        5:31,
        6:30,
        7:31,
        8:31,
        9:30,
        10:31,
        11:30,
        12:26
    }
    for mjesec in danaUMjesecu.keys():

        for dan in range(1, danaUMjesecu[mjesec] + 1):
            datum = "2020-"
            if mjesec < 10:
                datum += "0%d-" % mjesec
            else:
                datum += str(mjesec) + "-"

            if dan < 10:
                datum += "0%d" % dan
            else:
                datum += str(dan)
            listaDatuma.append(datum)
    listaDatuma = listaDatuma[4:]
    return listaDatuma

In [38]:
lisatDatuma = getDatumi()
posmatrani_gradovi = ['Banja Luka', 'Prijedor', # 'Gradiška',
                      'Biljejina', 'Trebinje', 'Doboj']
stavke = ["novi","umrli","oporavljeni","testirani","kriticni"]
gradovi_skraceni_zapis = {
    'Banja Luka': 'BL',
    'Prijedor': 'PRI',
    'Gradiška': 'GR',
    'Biljejina':'BN',
    'Trebinje':'TB',
    'Doboj':'DB'
}

In [39]:
def getPodaciDatum(datum):
    url_ = "https://covid19.etfbl.net:8443/api/public/dnevni-podaci/by-date/%s" % datum
    page = requests.get(url_)
    #page.encoding = 'utf-8'
    filename_originalni = "..\\data-etfbl\\originalni-podaci\\%s.txt" % datum
    s=""
    with open(filename_originalni, "w", encoding='utf-8') as f:
        s = str(page.content)
        s = s[2:-1]
        s = s.replace("null", "-1")
        f.write(s.encode('utf8', 'surrogateescape').decode('utf8'))
    return eval(s)

In [40]:
def getPodaci():
    ucitaniPodaci = {}
    dataGrad = {'datum': [],
                'Banja Luka': {
                    'novi': [],
                    'umrli': [],
                    'oporavljeni': [],
                    'testirani': [],
                    'kriticni': []
                },
                'Prijedor': {
                    'novi': [],
                    'umrli': [],
                    'oporavljeni': [],
                    'testirani': [],
                    'kriticni': []
                },
                #'Gradiška': {
                #    'novi': [],
                #    'umrli': [],
                #    'oporavljeni': [],
                #    'testirani': [],
                #    'kriticni': []
                #},
                'Biljejina': {
                    'novi': [],
                    'umrli': [],
                    'oporavljeni': [],
                    'testirani': [],
                    'kriticni': []
                },
                'Trebinje': {
                    'novi': [],
                    'umrli': [],
                    'oporavljeni': [],
                    'testirani': [],
                    'kriticni': []
                },
                'Doboj': {
                    'novi': [],
                    'umrli': [],
                    'oporavljeni': [],
                    'testirani': [],
                    'kriticni': []
                }
            }
    for index_, datum in enumerate(lisatDatuma):
        podaciDatum = getPodaciDatum(datum)
        
        # Kreiranje datoteka po danu koji se mogu predstaviti u pandas
        filename_datum = "..\\data-etfbl\\%s.txt" % datum
        podaciDatumObrada = []
        for mjesto in podaciDatum:
            dan = {
                "Mjesto": mjesto["mjesto"]["nazivLat"],
                "Novi" : mjesto["novi"], 
                "Umrli" : mjesto["umrli"],
                "Oporavljeni" : mjesto["oporavljeni"],
                "Testirani" : mjesto["testirani"],
                "Kriticni" : mjesto["kriticni"]
            }
            podaciDatumObrada.append(dan)

        with open(filename_datum, "w", encoding='utf-8') as f:
            f.write(json.dumps(podaciDatumObrada))
        
        
        dataGrad['datum'].append(datum)
        for grad in podaciDatum:
            trenutni_grad = grad['mjesto']['nazivLat']
            if trenutni_grad in posmatrani_gradovi:
                for stavka in stavke:
                    dataGrad[trenutni_grad][stavka].append(grad[stavka])
                    
        for grad in posmatrani_gradovi:
            if len(dataGrad[grad]['novi']) < index_ + 1:
                for stavka in stavke:
                    dataGrad[grad][stavka].append(0)
    return dataGrad

In [41]:
res = getPodaci()
with open("..\\data-etfbl\\podaci_po_gradovima_2.txt", "w") as f:
    f.write(json.dumps(res))

In [42]:
pandas_data = [{"datum": datum} for datum in res['datum']]

pandas_data2 = { 
    "Datum" : res['datum']
    }

for grad in posmatrani_gradovi:
    for stavka in stavke:
        k = "%s %s" % (gradovi_skraceni_zapis[grad], stavka)
        v = res[grad][stavka]
        pandas_data2[k] = v
        for i, s in enumerate(v):
            pandas_data[i][k] = s

In [43]:
with open("..\\data-etfbl\\podaci_po_gradovima.txt", "w") as f:
    f.write(json.dumps(pandas_data))

In [44]:
pd_df_gradovi = pd.DataFrame(data=pandas_data)
pd_df_gradovi.head(25)

,datum,BL novi,BL umrli,BL oporavljeni,BL testirani,BL kriticni,PRI novi,PRI umrli,PRI oporavljeni,PRI testirani,...,TB novi,TB umrli,TB oporavljeni,TB testirani,TB kriticni,DB novi,DB umrli,DB oporavljeni,DB testirani,DB kriticni
0,2020-03-05,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-03-06,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-03-07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-03-08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-03-09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2020-03-10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2020-03-11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2020-03-12,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2020-03-13,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2020-03-14,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
